# Introduction to testing and py.test

## http://github.com/BCNDojos/pytest-intro

This notebook is intended to introduce a practice session on testing and py.test.

## Testing, TDD, and other stuff

* Testing is to guarantee a software works as expected
* This session is about **unit testing**
* Testing versus TDD
* Mocking

## Testing in Python

In [1]:
a = 1
b = 2
assert(a == 1)

In [2]:
assert(b == a)

AssertionError: 

## About py.test

* Created by [Holger Krekel](https://twitter.com/hpk42)
* Very pythonic, based in `assert`
* Integrates with other testing libraries in Python, like nosetest, unittest, ...
* Include many features, like mocking, test detection, fixtures, parametrization, ...

## Using py.test in a project

Summarizing, py.test will detect all tests according to:

* files are called `test_*.py`
* functions are called `test_*`
* classes are called `Test*`, running methods called `test_*`

 ## Example

In [3]:
%%bash
rm *example*

rm: *example*: No such file or directory


In [4]:
%%writefile division_example.py
def divide(x, y):
    pass

Writing division_example.py


In [5]:
%%writefile test_division_example.py
from division_example import divide

def test_something_right():
    assert divide(42, 1) == 42

Writing test_division_example.py


In [6]:
%%bash
py.test test_division_example.py

====================================================== test session starts =======================================================
platform darwin -- Python 3.4.4, pytest-2.9.1, py-1.4.31, pluggy-0.3.1
rootdir: /Users/ifosch/src/github.com/BCNDojos/pytest-intro, inifile: 
collected 1 items

test_division_example.py F

============================================================ FAILURES ============================================================
______________________________________________________ test_something_right ______________________________________________________

    def test_something_right():
>       assert divide(42, 1) == 42
E       assert None == 42
E        +  where None = divide(42, 1)

test_division_example.py:4: AssertionError
==================================================== 1 failed in 0.01 seconds ====================================================


In [7]:
%%writefile division_example.py
def divide(x, y):
    return x / y

Overwriting division_example.py


In [8]:
%%bash
py.test test_division_example.py

====================================================== test session starts =======================================================
platform darwin -- Python 3.4.4, pytest-2.9.1, py-1.4.31, pluggy-0.3.1
rootdir: /Users/ifosch/src/github.com/BCNDojos/pytest-intro, inifile: 
collected 1 items

test_division_example.py .

==================================================== 1 passed in 0.00 seconds ====================================================


## Testing exception raising

In [9]:
%%writefile test_division_example.py
import pytest
from division_example import divide

def test_something_right():
    assert divide(42, 1) == 42

def test_raises():
    with pytest.raises(ZeroDivisionError):
        divide(3, 0)

Overwriting test_division_example.py


In [10]:
%%bash
py.test test_division_example.py

====================================================== test session starts =======================================================
platform darwin -- Python 3.4.4, pytest-2.9.1, py-1.4.31, pluggy-0.3.1
rootdir: /Users/ifosch/src/github.com/BCNDojos/pytest-intro, inifile: 
collected 2 items

test_division_example.py ..

==================================================== 2 passed in 0.01 seconds ====================================================


## Parametrize

In [11]:
%%writefile test_param_division_example.py
import pytest
from division_example import divide

@pytest.mark.parametrize("a", [1, 2, 3, 4])
def test_division_param(a):
    assert divide(a, 1) == a

Writing test_param_division_example.py


In [12]:
%%bash
py.test test_param_division_example.py

====================================================== test session starts =======================================================
platform darwin -- Python 3.4.4, pytest-2.9.1, py-1.4.31, pluggy-0.3.1
rootdir: /Users/ifosch/src/github.com/BCNDojos/pytest-intro, inifile: 
collected 4 items

test_param_division_example.py ....

==================================================== 4 passed in 0.01 seconds ====================================================


In [13]:
%%writefile test_param_division_example.py
import pytest
from division_example import divide

@pytest.mark.parametrize("a", [1, 2, 3, 4])
def test_division_param(a):
    assert divide(a, 1) == a

@pytest.mark.parametrize("a", [(10, 2, 5), (100, 10, 10)])
def test_division_param_tuple(a):
    assert divide(a[0], a[1]) == a[2]

Overwriting test_param_division_example.py


In [14]:
%%bash
py.test test_param_division_example.py

====================================================== test session starts =======================================================
platform darwin -- Python 3.4.4, pytest-2.9.1, py-1.4.31, pluggy-0.3.1
rootdir: /Users/ifosch/src/github.com/BCNDojos/pytest-intro, inifile: 
collected 6 items

test_param_division_example.py ......

==================================================== 6 passed in 0.01 seconds ====================================================


## Monkeypatching

In [15]:
%%writefile env_reader_example.py
import os

def get_user():
    return os.environ['USER']

Writing env_reader_example.py


In [16]:
from env_reader_example import get_user

print(get_user())

ifosch


In [17]:
%%writefile test_env_reader_example.py
import os
from env_reader_example import get_user

def test_get_user():
    old = os.environ['USER']
    os.environ['USER'] = 'root'
    try:
        user = get_user()
        assert user == 'root'
    finally:
        os.environ['USER'] = old

Writing test_env_reader_example.py


In [18]:
%%bash
py.test test_env_reader_example.py

====================================================== test session starts =======================================================
platform darwin -- Python 3.4.4, pytest-2.9.1, py-1.4.31, pluggy-0.3.1
rootdir: /Users/ifosch/src/github.com/BCNDojos/pytest-intro, inifile: 
collected 1 items

test_env_reader_example.py .

==================================================== 1 passed in 0.00 seconds ====================================================


In [19]:
from env_reader_example import get_user

print(get_user())

ifosch


### Using old-style JUnit tests

In [20]:
%%writefile test_env_reader_example.py
import os
from env_reader_example import get_user

class TestGetUser:
    def setup_method(self, method):
        self._oldenv = os.environ.copy()
 
    def teardown_method(self, method):
        os.environ.clear()
        os.environ.update(self._oldenv)
 
    def test_envreading(self):
        os.environ['USER'] = "root"
        user = get_user()
        assert user == "root"

Overwriting test_env_reader_example.py


In [21]:
%%bash
py.test test_env_reader_example.py

====================================================== test session starts =======================================================
platform darwin -- Python 3.4.4, pytest-2.9.1, py-1.4.31, pluggy-0.3.1
rootdir: /Users/ifosch/src/github.com/BCNDojos/pytest-intro, inifile: 
collected 1 items

test_env_reader_example.py .

==================================================== 1 passed in 0.01 seconds ====================================================


In [22]:
from env_reader_example import get_user

print(get_user())

ifosch


### Well-done py.test monkeypatching

In [23]:
%%writefile test_env_reader_example.py
import os
from env_reader_example import get_user

def test_get_user(monkeypatch):
    monkeypatch.setitem(os.environ, 'USER', 'root')
    user = get_user()
    assert user == "root"

Overwriting test_env_reader_example.py


In [24]:
%%bash
py.test test_env_reader_example.py

====================================================== test session starts =======================================================
platform darwin -- Python 3.4.4, pytest-2.9.1, py-1.4.31, pluggy-0.3.1
rootdir: /Users/ifosch/src/github.com/BCNDojos/pytest-intro, inifile: 
collected 1 items

test_env_reader_example.py .

==================================================== 1 passed in 0.01 seconds ====================================================


In [25]:
from env_reader_example import get_user

print(get_user())

ifosch


## Monkeypatching (or mocking) other stuff

### Creating an example sqlite3 database

In [26]:
%%writefile gen_database.py
import sqlite3

conn = sqlite3.connect('example.db')
c = conn.cursor()
c.execute('''CREATE TABLE stocks ( date text, symbol text, qty real, price real)''')
c.execute("INSERT INTO stocks VALUES ('2006-01-05','RHAT',100,35.14)")
purchases = [('2006-01-05','RHAT',100,35.14),
             ('2006-03-28', 'IBM', 1000, 45.00),
             ('2006-04-05', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'IBM3', 500, 53.00),
            ]
c.executemany('INSERT INTO stocks VALUES (?,?,?,?)', purchases)
conn.commit()
conn.close()

Writing gen_database.py


In [27]:
%%bash
python gen_database.py

### Usage of the database

In [28]:
import sqlite3
conn = sqlite3.connect('example.db')
c = conn.cursor()
t = ('RHAT',)
c.execute('SELECT * FROM stocks WHERE symbol=?', t)
print(c.fetchone())
conn.close()

('2006-01-05', 'RHAT', 100.0, 35.14)


In [29]:
conn = sqlite3.connect('example.db')
c = conn.cursor()
c.execute('SELECT * FROM stocks')
print(c.fetchall())
conn.close()

[('2006-01-05', 'RHAT', 100.0, 35.14), ('2006-01-05', 'RHAT', 100.0, 35.14), ('2006-03-28', 'IBM', 1000.0, 45.0), ('2006-04-05', 'MSFT', 1000.0, 72.0), ('2006-04-06', 'IBM3', 500.0, 53.0)]


### Code for our database

In [30]:
%%writefile data_example.py
import sqlite3

def get_total():
    conn = sqlite3.connect('example.db')
    c = conn.cursor()
    c.execute('SELECT * FROM stocks')
    total = 0
    for item in c.fetchall():
        total += item[2] * item[3]
    return total

Writing data_example.py


In [31]:
from data_example import get_total

print(get_total())

150528.0


### Test for our code

    import sqlite3
    from data_example import get_total

    class FakeConn():
        def cursor(self):
            return FakeCursor()

    class FakeCursor():
        def execute(self, query):
            pass

        def fetchall(self):
            return  [
                        ('2006-01-05', 'RHAT', 1.0, 10.0),
                        ('2006-03-28', 'IBM', 1.0, 10.0),
                        ('2006-04-05', 'MSFT', 1.0, 10.0),
                        ('2006-04-06', 'IBM3', 1.0, 10.0)
                    ]

    def test_get_total(monkeypatch):
        def mock_conn(file_name):
            return FakeConn()
        monkeypatch.setattr(sqlite3, 'connect', mock_conn)
        total = get_total()
        assert total == 40.0

In [32]:
%%writefile test_data_example.py
import sqlite3
from data_example import get_total

class FakeConn():
    def cursor(self):
        return FakeCursor()

class FakeCursor():
    def execute(self, query):
        pass
    
    def fetchall(self):
        return  [
                    ('2006-01-05', 'RHAT', 1.0, 10.0),
                    ('2006-03-28', 'IBM', 1.0, 10.0),
                    ('2006-04-05', 'MSFT', 1.0, 10.0),
                    ('2006-04-06', 'IBM3', 1.0, 10.0)
                ]

def test_get_total(monkeypatch):
    def mock_conn(file_name):
        return FakeConn()
    monkeypatch.setattr(sqlite3, 'connect', mock_conn)
    total = get_total()
    assert total == 40.0

Writing test_data_example.py


In [33]:
%%bash
py.test test_data_example.py

====================================================== test session starts =======================================================
platform darwin -- Python 3.4.4, pytest-2.9.1, py-1.4.31, pluggy-0.3.1
rootdir: /Users/ifosch/src/github.com/BCNDojos/pytest-intro, inifile: 
collected 1 items

test_data_example.py .

==================================================== 1 passed in 0.01 seconds ====================================================


## Basic mocking of I/O

In [34]:
%%writefile user_input_example.py

def salute():
    name = input("What is your name? ")
    return 'Hello, {}'.format(name)

if __name__ == '__main__':
    print(salute())

Writing user_input_example.py


In [35]:
%%writefile test_user_input_example.py
from user_input_example import salute

def mock_input(message):
    return 'Mark'

def test_user_input(monkeypatch):
    monkeypatch.setitem(__builtins__, 'input', mock_input)
    message = salute()
    assert message == "Hello, Mark"

Writing test_user_input_example.py


In [36]:
%%bash
py.test test_user_input_example.py

====================================================== test session starts =======================================================
platform darwin -- Python 3.4.4, pytest-2.9.1, py-1.4.31, pluggy-0.3.1
rootdir: /Users/ifosch/src/github.com/BCNDojos/pytest-intro, inifile: 
collected 1 items

test_user_input_example.py .

==================================================== 1 passed in 0.01 seconds ====================================================


In [37]:
%%writefile user_input_example.py

def salute():
    name = input("What is your name? ")
    return 'Hello, {}'.format(name)

def print_salute():
    print(salute())

if __name__ == '__main__':
    print_salute()

Overwriting user_input_example.py


In [38]:
%%writefile test_user_input_example.py
from user_input_example import salute, print_salute

def mock_input(message):
    return 'Mark'

def test_user_input(monkeypatch, capsys):
    monkeypatch.setitem(__builtins__, 'input', mock_input)
    message = salute()
    assert message == "Hello, Mark"
    print_salute()
    out, err = capsys.readouterr()
    assert out == "Hello, Mark\n"

Overwriting test_user_input_example.py


In [39]:
%%bash
py.test test_user_input_example.py

====================================================== test session starts =======================================================
platform darwin -- Python 3.4.4, pytest-2.9.1, py-1.4.31, pluggy-0.3.1
rootdir: /Users/ifosch/src/github.com/BCNDojos/pytest-intro, inifile: 
collected 1 items

test_user_input_example.py .

==================================================== 1 passed in 0.01 seconds ====================================================


## Time enough for an exercise?

Let's make groups of two or three to develop a layered application in parallel, so the group will work in two different modules.

A half of the group will implement a Python module, let's call it `data` that reads from the database and gets stock price at a user specified date.

The other half will implement a text interface program that will ask for a date until it gets an EOF, and will use this date to get the value from the previous module.

Notice date is treated as a text in the database, and this is not the subject of this session, so no need to format dates, let's just consider user input is right.

In both cases, the trick is in to implement tests implementing appropriate mocking of whatever is required.